# Analysis of Spotify Tracks with Neural Networks
**A project by Graham Waters, prepared 2022-11-14**

Executive Summary

# Executive Summary


 This project is a continuation of the previous project, which was a classification problem. The goal of this project is to use the same data set and use clustering to identify potential genres that are not named after traditional genres but are generated by the algorithm. 


# Part 1: Using Clustering on Spotify data to predict genres of songs


We have a labeled dataset of spotify songs that have been labeled with a genre. In the first part of this project we want to use the features of the songs to predict the genre. We will use several methods, culminating in the use of a neural network to predict the genre. 


# Libraries we will use in this study


 ```python import pandas as pd import numpy as np import matplotlib.pyplot as plt import seaborn as sns import scipy.stats as stats import sklearn from sklearn.preprocessing import StandardScaler from sklearn.preprocessing import OneHotEncoder from sklearn.model_selection import train_test_split from sklearn.ensemble import GradientBoostingClassifier from sklearn.ensemble import RandomForestClassifier from sklearn.metrics import accuracy_score from sklearn.metrics import confusion_matrix from sklearn.metrics import classification_report from sklearn.metrics import plot_confusion_matrix from sklearn import svm from sklearn.cluster import DBSCAN 


# for neural networks


import tensorflow as tf from tensorflow import keras from tensorflow.keras import layers from tensorflow.keras.layers.experimental import preprocessing  ``` 


# What Data is available in our datasets?

## Data Dictionary

### File 1. data_by_genres.csv


 | Column Name | Description |
|-------------|-------------|
| genre | The genre of the song |
| acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
| danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
| duration_ms | The duration of the track in milliseconds. |
| energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
| instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
| key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
| liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
| loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
| mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
| speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
| tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
| time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
| valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/* 


### File 2. spotify_artists.csv


  | Column Name | Description |
|-------------|-------------|
| artist_popularity | The popularity of the artist. The value will be between 0 and 100, with 100 being the most popular. The artist’s popularity is calculated from the popularity of all the artist’s tracks. |
| followers | The number of followers the artist has. |
| genres | A list of the genres the artist is associated with. For example: “Prog Rock” , “Post-Grunge”. (If not yet classified, the array is empty.) |
| id | The Spotify ID for the artist. |
| name | The name of the artist. |
| track_id | The Spotify ID for the track. |
| track_name_prev | The name of the track. |
| type | The object type: “artist” |  *source: https://developer.spotify.com/documentation/web-api/reference/artists/get-artist/* 


### File 3. spotify_tracks.csv


  | Column Name | Description |
|-------------|-------------|
| acousticness | A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic. |
| album_id | The Spotify ID for the album. |
| analysis_url | An HTTP URL to access the full audio analysis of this track. An access token is required to access this data. |
| artists_id | The Spotify ID for the artist. |
| available_markets | A list of the countries in which the track can be played, identified by their ISO 3166-1 alpha-2 code. |
| country | The country of the album, identified by its ISO 3166-1 alpha-2 code. |
| danceability | Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable. |
| disc_number | The disc number (usually 1 unless the album consists of more than one disc). |
| duration_ms | The duration of the track in milliseconds. |
| energy | Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy. |
| href | A link to the Web API endpoint providing full details of the track. |
| id | The Spotify ID for the track. |
| instrumentalness | Predicts whether a track contains no vocals. “Ooh” and “aah” sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly “vocal”. The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0. |
| key | The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. |
| liveness | Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live. |
| loudness | The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db. |
| lyrics | The lyrics of the track. |
| mode | Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0. |
| name | The name of the track. |
| playlist | The name of the playlist. |
| popularity | The popularity of the track. The value will be between 0 and 100, with 100 being the most popular. The popularity of a track is a value between 0 and 100, with 100 being the most popular. The popularity is calculated by algorithm and is based, in the most part, on the total number of plays the track has had and how recent those plays are. Generally speaking, songs that are being played a lot now will have a higher popularity than songs that were played a lot in the past. Duplicate tracks (e.g. the same track from a single and an album) are rated independently. Artist and album popularity is derived mathematically from track popularity. Note that the popularity value may lag actual popularity by a few days: the value is not updated in real time. |
| preview_url | A link to a 30 second preview (MP3 format) of the track. |
| speechiness | Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks. |
| tempo | The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration. |
| time_signature | An estimated overall time signature of a track. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). |
| track_href | A link to the Web API endpoint providing full details of the track. |
| track_name_prev | The name of the track. |
| track_number | The number of the track. If an album has several discs, the track number is the number on the specified disc. |
| uri | The Spotify URI for the track. |
| valence | A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). |
| type | The object type: “track” |  *source: https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/* 


### File 4. low_level_audio_features.csv


   | Column Name | Description |
|-------------|-------------|
| Chroma_1 | Chroma feature 1 |
| Chroma_10 | Chroma feature 10 |
| Chroma_11 | Chroma feature 11 |
| Chroma_12 | Chroma feature 12 |
| Chroma_2 | Chroma feature 2 |
| Chroma_3 | Chroma feature 3 |
| Chroma_4 | Chroma feature 4 |
| Chroma_5 | Chroma feature 5 |
| Chroma_6 | Chroma feature 6 |
| Chroma_7 | Chroma feature 7 |
| Chroma_8 | Chroma feature 8 |
| Chroma_9 | Chroma feature 9 |
| MFCC_1 | MFCC feature 1 |
| MFCC_10 | MFCC feature 10 |
| MFCC_11 | MFCC feature 11 |
| MFCC_12 | MFCC feature 12 |
| MFCC_13 | MFCC feature 13 |
| MFCC_14 | MFCC feature 14 |
| MFCC_15 | MFCC feature 15 |
| MFCC_16 | MFCC feature 16 |
| MFCC_17 | MFCC feature 17 |
| MFCC_18 | MFCC feature 18 |
| MFCC_19 | MFCC feature 19 |
| MFCC_2 | MFCC feature 2 |
| MFCC_20 | MFCC feature 20 |
| MFCC_3 | MFCC feature 3 |
| MFCC_4 | MFCC feature 4 |
| MFCC_5 | MFCC feature 5 |
| MFCC_6 | MFCC feature 6 |
| MFCC_7 | MFCC feature 7 |
| MFCC_8 | MFCC feature 8 |
| MFCC_9 | MFCC feature 9 |
| Tonnetz_1 | Tonnetz feature 1 |
| Tonnetz_2 | Tonnetz feature 2 |
| Tonnetz_3 | Tonnetz feature 3 |
| Tonnetz_4 | Tonnetz feature 4 |
| Tonnetz_5 | Tonnetz feature 5 |
| Tonnetz_6 | Tonnetz feature 6 |
| ZCR | Zero Crossing Rate |
| entropy_energy | Entropy of the energy |
| spectral_bandwith | Spectral Bandwith |
| spectral_centroid | Spectral Centroid |
| spectral_rollOff_max | Spectral RollOff Max |
| spectral_rollOff_min | Spectral RollOff Min |
| track_id | Spotify track ID |  *source:* 


### File 5. track_features.csv


   | Column Name | Description |
|-------------|-------------|
| mean_syllables_word | Mean number of syllables per word |
| mean_words_sentence | Mean number of words per sentence |
| n_sentences | Number of sentences |
| n_words | Number of words |
| sentence_similarity | Sentence similarity |
| track_id | Spotify track ID |
| vocabulary_wealth | Vocabulary wealth |  *source:* 


## Comments about the Next Steps

### Get rid of columns that don't help predict genre

### Look into the pandas join function


 The end goal is to have single dataframe joining five tables on the track id column note: check if track id exists in all tables 


## EDA - Exploratory Data Analysis


Check for the trends in the data to see the correlations, skewness and kurtosis of the data, how they relate to dependent variable, and how they relate to each other.  


### Loading The Data from the Files


Using pandas to load the data from the files into dataframes.  ```python


# import the data from the csv files into pandas dataframes (data/SpotGenTrack)



Section Text Not Found



# FeaturesExtracted from the audio files


df_features = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/lyrics_features.csv') df_lowlevelaudio = pd.read_csv('data/SpotGenTrack/FeaturesExtracted/low_level_audio_features.csv') 


# print the columns for each data file to see what data is available


print(f'\ncolumns in the albums file: ',df_albums.columns) print(f'\ncolumns in the artists file: ',df_artists.columns) print(f'\ncolumns in the tracks file: ',df_tracks.columns) print(f'\ncolumns in the features file: ',df_features.columns) print(f'\ncolumns in the lowlevelaudio file: ',df_lowlevelaudio.columns) ```  ```output columns in the albums file:  Index(['Unnamed: 0', 'album_type', 'artist_id', 'available_markets',        'external_urls', 'href', 'id', 'images', 'name', 'release_date',        'release_date_precision', 'total_tracks', 'track_id', 'track_name_prev',        'uri', 'type'],       dtype='object')  columns in the artists file:  Index(['Unnamed: 0', 'artist_popularity', 'followers', 'genres', 'id', 'name',        'track_id', 'track_name_prev', 'type'],       dtype='object')  columns in the tracks file:  Index(['Unnamed: 0', 'acousticness', 'album_id', 'analysis_url', 'artists_id',        'available_markets', 'country', 'danceability', 'disc_number',        'duration_ms', 'energy', 'href', 'id', 'instrumentalness', 'key',        'liveness', 'loudness', 'lyrics', 'mode', 'name', 'playlist',        'popularity', 'preview_url', 'speechiness', 'tempo', 'time_signature',        'track_href', 'track_name_prev', 'track_number', 'uri', 'valence',        'type'],       dtype='object')  columns in the features file:  Index(['Unnamed: 0', 'mean_syllables_word', 'mean_words_sentence',        'n_sentences', 'n_words', 'sentence_similarity', 'track_id',        'vocabulary_wealth'],       dtype='object') ...        'Tonnetz_4', 'Tonnetz_5', 'Tonnetz_6', 'ZCR', 'entropy_energy',        'spectral_bandwith', 'spectral_centroid', 'spectral_rollOff_max',        'spectral_rollOff_min', 'track_id'],       dtype='object', length=209) ``` 


### What are the most popular genres?


 ```python


# get the counts of each genre


genre_counts = df_tracks['playlist'].value_counts() ```


### What are the most popular artists?


 ```python


# get the counts of each artist


artist_counts = df_tracks['artists_id'].value_counts() ``` 


### What are the most popular tracks?


 ```python


# get the counts of each track


track_counts = df_tracks['id'].value_counts() ``` 


### What are the most popular albums?


 ```python


# get the counts of each album


album_counts = df_tracks['album_id'].value_counts() ``` 


### What are the most popular countries?


 ```python


# get the counts of each country


country_counts = df_tracks['country'].value_counts() ```   


### What correlations are there between the features?


 To find the correlations we will likely want to use a test for multicollinearity to see if there are any features that are highly correlated with each other. If there are, we will want to remove one of the features from the model.   ```python


# Testing for multicollinearity

# create a correlation matrix


corr_matrix = df_tracks.corr() # .corr is used to find corelation


# plot the correlation matrix


plt.figure(figsize=(20,20)) sns.heatmap(corr_matrix, annot=True, cmap='coolwarm') plt.show() ``` 


### What are the distributions of the features?


 ```python


# plot the distributions of the features


df_tracks.hist(figsize=(20,20)) plt.show() ``` 


### How do each of the features relate to the target (genre)?


  | Part One | Part Two |
|----------|----------|
| [Part One](#part-one) | [Part Two](#part-two) |
| Classification with NNets | Clustering with DBScan| 


## Data Preprocessing

### Missing data

### types of data (categorical, numerical, ordinal)



Section Text Not Found



### standardize the columns (scale the data).



Section Text Not Found



### Encode the categorical data (one hot encoding) use get dummies (vs one hot encoding)



Section Text Not Found



## Start building the models

### Gradient Boosting Model


 A general purpose machine learning algorithm that can be used for both regression and classification problems. I could compare XGBoost to Gradient Boosting Model. It is a package in python that is an implementation of gradient boosting. 


### Random Forest Model

### Support Vector Machine potentially.

### Use A neural network (deep learning)



Section Text Not Found



## Evaluate the models


Any of the metrics below could be worth looking at when scoring how well our models do when dealing with the data. * accuracy * precision * recall * f1 score * confusion matrix * ROC curve * AUC score  --- 


# Part 2. Genre Generation with DBSSCAN Clustering


Remove Genres from the data and using DBSCAN to cluster the data and identify potential genres that are not named after traditional genres but are generated by the algorithm. 


# Components to this section of the project:



Section Text Not Found

